# Парсим torg.mail.ru 

requests и bs4 - для парсинга страниц и выделения из запросов нужного текста, time и random - для ожидания между запросами, pandas - для сохранения финальных результатов, join - для работы с путями.

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from time import sleep
from numpy.random import random as rnd
import pandas as pd
from os.path import join as pjoin

Функции для добавления отзывов и рейтингов в списки.

In [2]:
def add_new_reviews(reviews, request_output):
    new_reviews = [element.text for element in request_output]
    reviews += new_reviews
    return reviews

def add_new_ratings(ratings, request_output):
    new_ratings = [int(element.text) for element in request_output]
    ratings += new_ratings
    return ratings

Нужен источник отзывов. Воспользуемся разделом отзывов на телефоны сайта torg.mail.ru - его удобно парсить минимумом усилий, не используя модуль scrapy, а отзывов достаточно много. К тому же, при желании, можно столь же легко парсить и несколько разделов. И никаког api не нужно!

In [3]:
start_url = 'https://torg.mail.ru/review/goods/mobilephones/?page={}'

In [4]:
reviews = []
ratings = []

In [5]:
number_of_pages = 1000 
# На самом деле, страниц меньше, чем 1000, точнее страниц может быть сколько угодно, но они пустые.

In [6]:
for page in range(number_of_pages):
    sleep(rnd()) # Имитируем нормальное повежение.
    req = requests.get(start_url.format(page+2))
    parser = bs(req.text, 'lxml')
    x = parser.findAll('div', attrs={'class': 'review-item__content'})
    y = parser.findAll('span', attrs={'class': 'review-item__rating-counter'})
    reviews = add_new_reviews(reviews, x)
    ratings = add_new_ratings(ratings, y) 
    print("{} pages parsed.".format(page+1))

1 pages parsed.
2 pages parsed.
3 pages parsed.
4 pages parsed.
5 pages parsed.
6 pages parsed.
7 pages parsed.
8 pages parsed.
9 pages parsed.
10 pages parsed.


In [7]:
print("Results: {pages} pages parsed, with {reviews} reviews.".format(pages=page+1, reviews=len(reviews)))

Results: 10 pages parsed, with 0 reviews.


На случай, если захочется посмотреть, что же там в отзывах.

In [8]:
def review_rating_info(reviews, ratings, length=5, start=0):
    for index in range(length):
        print(reviews[start+index])
        print("Rating: {}\n".format(ratings[start+index]))

In [9]:
review_rating_info(reviews, ratings)

IndexError: list index out of range

In [ ]:
df = pd.DataFrame(data={'review': reviews, 'rating': ratings})
df.head()

In [ ]:
out_path = pjoin('data', 'reviews.csv')
df.to_csv(out_path, encoding='utf-8', index=False)

In [ ]:
df0 = pd.read_csv(out_path, encoding='utf-8')
df0.head()